In [40]:
import pandas as pd

df = pd.read_csv("../data/online_gaming_behavior_dataset.csv")
df.describe()

,PlayerID,Age,PlayTimeHours,InGamePurchases,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked
count,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000
mean,29016.500000,31.992531,12.024365,0.200854,9.471774,94.792252,49.655568,24.526477
std,11556.964675,10.043227,6.914638,0.400644,5.763667,49.011375,28.588379,14.430726
min,9000.000000,15.000000,0.000115,0.000000,0.000000,10.000000,1.000000,0.000000
25%,19008.250000,23.000000,6.067501,0.000000,4.000000,52.000000,25.000000,12.000000
50%,29016.500000,32.000000,12.008002,0.000000,9.000000,95.000000,49.000000,25.000000
75%,39024.750000,41.000000,17.963831,0.000000,14.000000,137.000000,74.000000,37.000000
max,49033.000000,49.000000,23.999592,1.000000,19.000000,179.000000,99.000000,49.000000


In [155]:
df

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40029,49029,32,Male,USA,Strategy,20.619662,0,Easy,4,75,85,14,Medium
40030,49030,44,Female,Other,Simulation,13.539280,0,Hard,19,114,71,27,High
40031,49031,15,Female,USA,RPG,0.240057,1,Easy,10,176,29,1,High
40032,49032,34,Male,USA,Sports,14.017818,1,Medium,3,128,70,10,Medium


In [167]:
y = df["EngagementLevel"]
X = df.drop(["PlayerID", "EngagementLevel"], axis=1)

In [168]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher

enc = LabelEncoder()
hasher = FeatureHasher(n_features=2, input_type="string")
hashed_features = hasher.transform(X["Gender"].apply(lambda x: [x]))
hashed_df = pd.DataFrame(hashed_features.toarray(), columns=[f'gender_{i}' for i in range(2)])
X = pd.concat([X, hashed_df], axis=1)
X["Location"] = enc.fit_transform(X["Location"])
X["GameGenre"] = enc.fit_transform(X["GameGenre"])
X["GameDifficulty"] = enc.fit_transform(X["GameDifficulty"])
X["PlayTimePerWeek"] = X["SessionsPerWeek"] * X["AvgSessionDurationMinutes"] / 60
X = X.drop(["SessionsPerWeek", "AvgSessionDurationMinutes", "Gender"], axis=1)
y_enc = enc.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.33, random_state=42)

In [171]:
X.describe()

,Age,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,PlayerLevel,AchievementsUnlocked,gender_0,gender_1,PlayTimePerWeek
count,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000,40034.00000,40034.000000,40034.000000,40034.000000,40034.000000,40034.000000
mean,31.992531,1.695409,2.001049,12.024365,0.200854,0.80007,49.655568,24.526477,0.598466,0.401534,14.961261
std,10.043227,1.189780,1.415431,6.914638,0.400644,0.87179,28.588379,14.430726,0.490215,0.490215,12.854674
min,15.000000,0.000000,0.000000,0.000115,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,1.000000,1.000000,6.067501,0.000000,0.00000,25.000000,12.000000,0.000000,0.000000,4.466667
50%,32.000000,1.000000,2.000000,12.008002,0.000000,1.00000,49.000000,25.000000,1.000000,0.000000,11.400000
75%,41.000000,3.000000,3.000000,17.963831,0.000000,2.00000,74.000000,37.000000,1.000000,1.000000,22.733333
max,49.000000,3.000000,4.000000,23.999592,1.000000,2.00000,99.000000,49.000000,1.000000,1.000000,56.683333


In [74]:
from sklearn.metrics import classification_report

def train_evaluate(clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_pred,y_test))

In [111]:
from sklearn.svm import SVC

clf = SVC()
train_evaluate(clf)

              precision    recall  f1-score   support

           0       0.84      0.93      0.89      3118
           1       0.90      0.90      0.90      3343
           2       0.95      0.90      0.92      6751

    accuracy                           0.91     13212
   macro avg       0.90      0.91      0.90     13212
weighted avg       0.91      0.91      0.91     13212



In [169]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
train_evaluate(clf)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      3291
           1       0.90      0.92      0.91      3287
           2       0.95      0.92      0.94      6634

    accuracy                           0.92     13212
   macro avg       0.91      0.92      0.92     13212
weighted avg       0.92      0.92      0.92     13212



In [170]:
from sklearn.neural_network import MLPClassifier

cfl = MLPClassifier()
train_evaluate(clf)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      3289
           1       0.90      0.92      0.91      3287
           2       0.95      0.92      0.94      6636

    accuracy                           0.92     13212
   macro avg       0.91      0.92      0.92     13212
weighted avg       0.92      0.92      0.92     13212



In [115]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion="entropy")
train_evaluate(clf)

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      3479
           1       0.83      0.82      0.82      3400
           2       0.88      0.89      0.88      6333

    accuracy                           0.85     13212
   macro avg       0.85      0.84      0.85     13212
weighted avg       0.85      0.85      0.85     13212



In [106]:
import tensorflow as tf

#create model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='softmax'),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)

#evalate model
model.evaluate(X_test, y_test)


Epoch 1/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7399 - loss: 0.8758
Epoch 2/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8434 - loss: 0.4616
Epoch 3/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8510 - loss: 0.4442
Epoch 4/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8640 - loss: 0.4256
Epoch 5/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8719 - loss: 0.4155
Epoch 6/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8782 - loss: 0.4096
Epoch 7/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8873 - loss: 0.3911
Epoch 8/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8876 - loss: 0.3898
Epoch 9/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8905 - loss: 0.3878
Epoch 10/10
839/839 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8913 - loss: 0.3839
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8932 - loss: 0.3950


[0.38349056243896484, 0.8971389532089233]

In [107]:
#classification report
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)
print(classification_report(y_pred,y_test))

413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      3368
           1       0.85      0.92      0.88      3097
           2       0.94      0.89      0.91      6747

    accuracy                           0.90     13212
   macro avg       0.89      0.90      0.89     13212
weighted avg       0.90      0.90      0.90     13212

